In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

In [20]:
# stacked lstm
class LSTM(nn.Module):
    def __init__(self, n_hidden=64):
        super(LSTM, self).__init__()
        self.n_hidden = n_hidden
        
        # layers
        self.lstm1 = nn.LSTMCell(1, self.n_hidden)
        self.lstm2 = nn.LSTMCell(self.n_hidden, self.n_hidden)
        self.linear = nn.Linear(self.n_hidden, 1)
        
        
    def forward(self, x, future=0):
        outputs = []
        h_t = torch.zeros(x.size(0), self.n_hidden, dtype=torch.float32)
        c_t = torch.zeros(x.size(0), self.n_hidden, dtype=torch.float32)
        h_t2 = torch.zeros(x.size(0), self.n_hidden, dtype=torch.float32)
        c_t2 = torch.zeros(x.size(0), self.n_hidden, dtype=torch.float32)

        for input_t in x.split(1, dim=1): # .split(1, dim=1)
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.cat(outputs, dim=1)
        return outputs

In [21]:
inputs = np.array([[200],[400],[600],[800],[1000],[1200],[1400],[1600],[1800],[2000]], dtype=np.float32)
target = np.array([[80],[160],[240],[320],[400],[480.0], [560.0], [640.0], [720.0], [800.0]], dtype=np.float32)

sc1 = MinMaxScaler(feature_range=(0,1))
sc2 = MinMaxScaler(feature_range=(0,1))

sc1.fit(inputs)
sc2.fit(target)

inputs = sc1.transform(inputs)
target = sc2.transform(target)

train_input = torch.from_numpy(inputs)   # [7,1], [0:7]
train_target = torch.from_numpy(target)  # [7,1], [0:7]
test_input = torch.from_numpy(inputs)    # [3,1], [7:]
test_target = torch.from_numpy(target)   # [3,1], [7:]

In [22]:
model = LSTM()

criterion = nn.MSELoss()
optimizer = optim.LBFGS(model.parameters(), lr=0.01)

n_steps = 500

# training loop
for i in range(n_steps):    
    def closure():
        # zero the gradients
        optimizer.zero_grad()
        
        # forward pass
        out = model(train_input)
        
        # calculate the loss
        loss = criterion(out, train_target)
        
        # backward pass
        loss.backward()
        return loss
    
    optimizer.step(closure) # update the parameters
    
    with torch.no_grad():
        future = 1
        pred = model(test_input, future=future)
        loss = criterion(pred[:, :-1], test_target)
        y = pred.detach().numpy()
        
        if i % 50 == 0:
            print(f"Loss in training: {loss.item()}")
            print(f"Test loss: {loss.item()}")
    
    
def predict(x):
    x_scaled = (x - 200) / (2000 - 200)
    y_actual = x * 0.4
    input_t = torch.tensor([[x_scaled]])
    pred = model(input_t)
    pred_val = pred.detach().numpy()
    y_scaled = pred_val.item() * (800 - 80) + 80
    print("x_scaled", x_scaled)
    print("pred", y_scaled)
    print("y_actual", y_actual)
    
predict(876)

# x_flatten = sc.inverse_transform(test_input).flatten()
# y_flatten = sc.inverse_transform(test_target).flatten()
# pred_flatten = sc.inverse_transform(y[:, :-1]).flatten()

# plt.figure(figsize=(12,6))
# plt.xlabel('X')
# plt.ylabel('Y')


# plt.plot(x_flatten, y_flatten, 'r')
# plt.plot(x_flatten, pred_flatten, 'b--')
# plt.show()

Loss in training: 0.2434960901737213
Test loss: 0.2434960901737213
Loss in training: 8.20362583908718e-06
Test loss: 8.20362583908718e-06
Loss in training: 7.309389729925897e-06
Test loss: 7.309389729925897e-06
Loss in training: 7.296432158909738e-06
Test loss: 7.296432158909738e-06
Loss in training: 7.291098881978542e-06
Test loss: 7.291098881978542e-06
Loss in training: 7.2889524744823575e-06
Test loss: 7.2889524744823575e-06
Loss in training: 7.2879834078776184e-06
Test loss: 7.2879834078776184e-06
Loss in training: 7.287332209671149e-06
Test loss: 7.287332209671149e-06
Loss in training: 7.2866400842031e-06
Test loss: 7.2866400842031e-06
Loss in training: 7.2860566433519125e-06
Test loss: 7.2860566433519125e-06
x_scaled 0.37555555555555553
pred 348.6456775665283
y_actual 350.40000000000003


In [24]:
# save the model
PATH = 'stacked_lstm.pt'
torch.save(model.state_dict(), PATH)

In [25]:
# load and export the model to onnx
trained_model = LSTM()
trained_model.load_state_dict(torch.load('stacked_lstm.pt'))
trained_model.eval()
dummy_input = torch.tensor([[0.01]])
torch.onnx.export(trained_model, dummy_input, 'stacked_lstm-new.onnx', verbose=True)